# Python3.9.12

## 使用套件

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from urllib.parse import quote
import re
import requests
import sys
import os
import time
import json
import html2text
import pandas as pd
import undetected_chromedriver as uc
from datetime import datetime
import json
from webdriver_manager.chrome import ChromeDriverManager

## 店家詳細資訊+評論

In [2]:
def initialize_chrome(_from="facebook",retry=0):
    global driver 
    try:
        print("Initializing chromedriver.")
        options = Options()
        options.add_argument("--start-maximized")
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")
        options.add_argument("--headless")
        options.add_argument('--log-level=3')
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        time.sleep(2)
        return True
    except Exception as e:
        print(e)
        pass

def parse_description(description_tag):
    h = html2text.HTML2Text()
    h.ignore_links = True
    description_text = h.handle(str(description_tag)) 
    return description_text

#review區域
def get_review():
    print('請輸入開始數字\n')
    num = int(input())
    while num =< 100:
        # 下一頁review內容的token(寫進url)
        next_page_token = ""

        # 設定biz_id(寫進url)
        url = df['google_url'][num]
        biz_id = re.search(r"1s(0.*?\:.*?)[^a-zA-Z\d\s:]", url) # 寫成迴圈的時候要改一下
        if not biz_id:
            print("Not a valid url.")
        biz_id = biz_id.groups()[0]
        place_name = df['place_name'][num]
        reviewData = []
        try:
            while True:
                url = f'https://www.google.com/async/reviewSort?yv=3&async=feature_id:{biz_id},review_source:All%20reviews,sort_by:newestFirst,is_owner:false,filter_text:,associated_topic:,next_page_token:{next_page_token},_pms:s,_fmt:json'
                response = requests.get(url)

                response = response.text.removeprefix(")]}'")

                json_data = json.loads(response)["localReviewsProto"]

                if json_data.get('other_user_review'):
                    review_data = json_data["other_user_review"]

                    for result in review_data:
                        personal_rating = result['star_rating']['value']
                        author_name = result['author_real_name']
                        review_date = result['publish_date']['localized_date']
                        review_acquisition_date = datetime.now().strftime("%Y-%m-%d")
                        if result.get('review_text'):
                            review_text = parse_description(result['review_text']['full_html'])
                        else:
                            review_text = ""

                        info_dict = {}        
                        info_dict['author_name'] = author_name
                        info_dict['personal_rating'] =  personal_rating             
                        info_dict["review_date"] = review_date
                        info_dict["review_text"] = review_text
                        info_dict["review_acquisition_date"] = review_acquisition_date
                        info_dict["google_url"] = url
                        info_dict["biz_id"] = biz_id
                        reviewData.append(info_dict)

                    next_page_token = json_data.get('next_page_token','').strip()
                    if not next_page_token:
                        break
                else:
                    break
        except:
            num+=1
            continue
        reviews_info_header = list(reviewData[0].keys())
        reviews_info_df = pd.DataFrame.from_records(reviewData,columns=reviews_info_header)
        reviews_info_df.to_csv(f'./data/reviews/restaurant/{str(num)}_{place_name.replace("/","")[0:6]}.csv')
        print(str(num) + ' 完成店名(review)' + place_name)
        num += 1

In [6]:
if __name__ == '__main__':
    print('請確定該py檔與url的cvs檔案放在相同路徑')
    file = input('請輸入url檔案名稱(含完整副檔名)，例如: place_info.csv\n')
    df = pd.read_csv(file)
    time1 = time.time()
    get_review()
    print(f'執行總花費時間: {time.time() - time1}')
    input('任務完成，若要關閉該視窗請輸入ENTER.....')

請確定該py檔與url的cvs檔案放在相同路徑
請輸入url檔案名稱(含完整副檔名)，例如: place_info.csv
info_final.csv
0 完成店名(review)黑曜麵屋
1 完成店名(review)樂山窄巷拉麵
2 完成店名(review)日本拉麵屋
3 完成店名(review)荷麵亭Hasumentei 北投店
4 完成店名(review)らあめん花月嵐 北投石牌店
5 完成店名(review)滿客屋拉麵
6 完成店名(review)滿足溫泉拉麵/北投拉麵/北投美食/北投餐廳/北投必吃//北投Restaurants/台北美食推薦
7 完成店名(review)長生塩人 北投車站
8 完成店名(review)葉櫻拉麵 Hazakura
9 完成店名(review)拉麵公子
10 完成店名(review)隱家拉麵 士林店
11 完成店名(review)志明牛肉拉麵 北投總店
12 完成店名(review)道樂拉麵 大北店
13 完成店名(review)厚恩創意拉麵
14 完成店名(review)豚將日本拉麵
15 完成店名(review)和屋烏龍麵
16 完成店名(review)荷麵亭Hasumentei 士林店
17 完成店名(review)滿瀚鵝肉拉麵
18 完成店名(review)道樂屋台
19 完成店名(review)鳥人拉麵-台灣總店TOTTO Ramen
20 完成店名(review)武藏野壽司烏龍麵


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
if __name__ == '__main__':
    print('請確定該py檔與url的cvs檔案放在相同路徑')
    file = input('請輸入url檔案名稱(含完整副檔名)，例如: place_info.csv\n')
    df = pd.read_csv(file)
    time1 = time.time()
    get_review()
    print(f'執行總花費時間: {time.time() - time1}')
    input('任務完成，若要關閉該視窗請輸入ENTER.....')

請確定該py檔與url的cvs檔案放在相同路徑
請輸入url檔案名稱(含完整副檔名)，例如: place_info.csv
info_final.csv
請輸入開始數字

20
20 完成店名(review)武藏野壽司烏龍麵
21 完成店名(review)道樂商店
22 完成店名(review)麵飯食堂
23 完成店名(review)海鮮拉麵清燉豬腳
24 完成店名(review)長生塩人 天母
25 完成店名(review)張吳記什錦麵
26 完成店名(review)吳八鍋
27 完成店名(review)豐年公園口大腸麵線
28 完成店名(review)吳家牛肉麵店
29 完成店名(review)紅燈籠
30 完成店名(review)好聚會日式料理
31 完成店名(review)前景排骨酥麵
32 完成店名(review)北投36私房麵
33 完成店名(review)河南手工扯麵
34 完成店名(review)尾田家オダヤ
37 完成店名(review)鐵主廚（原滿足屋）
38 完成店名(review)八金坊日式丼飯
39 完成店名(review)品欣牛肉麵
42 完成店名(review)豪季水餃專賣店
43 完成店名(review)真鶴日式海鮮料理


In [3]:
if __name__ == '__main__':
    print('請確定該py檔與url的cvs檔案放在相同路徑')
    file = input('請輸入url檔案名稱(含完整副檔名)，例如: place_info.csv\n')
    df = pd.read_csv(file)
    time1 = time.time()
    get_review()
    print(f'執行總花費時間: {time.time() - time1}')
    input('任務完成，若要關閉該視窗請輸入ENTER.....')

請確定該py檔與url的cvs檔案放在相同路徑
請輸入url檔案名稱(含完整副檔名)，例如: place_info.csv
info_final.csv
請輸入開始數字

44
44 完成店名(review)小歇
45 完成店名(review)阿雪餐飲
46 完成店名(review)無名乾麵 - 公嬤麵
47 完成店名(review)快快丼食堂
48 完成店名(review)北投老又新牛肉麵
49 完成店名(review)小廚川味牛肉麵
50 完成店名(review)魚酥羹
51 完成店名(review)彈芽麵
52 完成店名(review)丼布狸居食屋
53 完成店名(review)洪家班餃子館
54 完成店名(review)朋來客棧
55 完成店名(review)阿樺師滷肉飯
56 完成店名(review)鮮味福州乾拌麵
57 完成店名(review)左賀食堂-壽司日式料理
60 完成店名(review)錢金涮涮鍋
61 完成店名(review)這一小鍋 台北北護店
62 完成店名(review)丰明殿 第一涮涮鍋殿堂-明德殿
64 完成店名(review)海原日式涮涮鍋福德店
65 完成店名(review)初九麻辣食堂
66 完成店名(review)火鍋世家
67 完成店名(review)王朝迷您石頭火鍋
68 完成店名(review)薪寶鍋超值小火鍋
69 完成店名(review)火鍋世家 石牌尊爵店
70 完成店名(review)六媽小火鍋
73 完成店名(review)八方悅日式鍋物 北投永興店
74 完成店名(review)小陶鍋精緻小火鍋
75 完成店名(review)貳房 頂級鍋物
76 完成店名(review)呷火鍋
77 完成店名(review)喜喜鍋HiHiPot北投石牌店淡水線石牌美食小火鍋
78 完成店名(review)喜喜鍋HiHiPot北投奇岩店淡水線奇岩美食小火鍋
79 完成店名(review)享鍋鍋膳之家
80 完成店名(review)發記麻辣火鍋（總店）美食推薦（北投美食/北投火鍋/北投小吃/北投高CP餐廳/北投在地美食/火鍋/百元鍋物）
81 完成店名(review)鍋董日式涮涮鍋 新北投旗艦店
82 完成店名(review)掘臭臭鍋-北投人氣小火鍋|CP小火鍋|熱門小火鍋|平價鍋物|必吃小火鍋|平價鍋物
83 完成店名(re